In [172]:
##### Assemble geographies covered by EU FADN
# the EU FADN geographies span multiple NUTS regions in multiple countries
# this script creates one layer which is the regions included in the FADN

import os
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

In [173]:
##### Load data

# Get the current working directory
cd = os.path.dirname(os.getcwd())

# Import data
NUTS = gpd.read_file('/Users/carinamanitius/Documents/Data/Admin_Boundaries/EU_NUTS/NUTS_RG_20M_2021_3035.gpkg')

NUTS_match = pd.read_csv("/Users/carinamanitius/Documents/Data/Admin_Boundaries/EU_FADN/FADN_region_NUTS_codes.csv")

FADN_match = pd.read_csv(f'{cd}/Data/Correspondence_tables/EU_FADN.csv')

# Set save path
save_path = '/Users/carinamanitius/Documents/Data/Admin_Boundaries/EU_FADN/EU_FADN_2021_regions.shp'

In [174]:
##### Filter to regions included 

NUTS = NUTS[NUTS['NUTS_ID'].isin(NUTS_match['NUTS_ID'])]

# split into regions to merge and regions not to merge

list_to_merge = [
    "DE6","DEF","UKC","UKE","UKG","UKD","UKF","UKK","UKH","UKJ",
    "EL51","EL62","EL53","EL41","EL52","EL63","EL54","EL42","EL65","EL43","EL64",
    "PT11","PT16D","PT16E","PT16F","PT16G","PT16H","PT16I","PT16J","PT17","PT15","PT20","PT16B","PT18",
    "PY30",
    "FI1B1","FI193","FI194","FI1D7","SE1","SE31","FI1C","FI1D1","FI195","FI1D8","SE2","SE32","FI1D2","FI196","FI1D9","FI1D3","FI197","FI1D5",
    "HU32","HU33",
    "PL42","PL22","PL43","PL21","PL6","PL41","PL7","PL82","PL51","PL81","PL84","PL52","PL92",
    "HR02","HR05","HR06"
]

NUTS_to_merge = NUTS[NUTS['NUTS_ID'].isin(list_to_merge)]
NUTS_no_merge = NUTS[~NUTS['NUTS_ID'].isin(list_to_merge)]

In [175]:
NUTS_no_merge.columns

Index(['NUTS_ID', 'LEVL_CODE', 'CNTR_CODE', 'NAME_LATN', 'NUTS_NAME',
       'MOUNT_TYPE', 'URBN_TYPE', 'COAST_TYPE', 'geometry'],
      dtype='object')

In [176]:
##### Merge regions that need to be merged 

def merge_nuts_regions(gdf, nuts_ids, fadn_code, col_to_keep=['FADN_code_2020', 'geometry']):

    subset = gdf[gdf['NUTS_ID'].isin(nuts_ids)]
    
    # Dissolve
    merged = subset.dissolve(as_index=False)
    merged = merged.reset_index(drop=True)

    # Add FADN code
    merged['FADN_code_2020'] = fadn_code

    # Keep only required columns
    merged = merged.loc[:, col_to_keep]

    return merged

# Germany
DE_15 = merge_nuts_regions(NUTS_to_merge, ['DE6', 'DEF'], 15)

# UK
UK_411 = merge_nuts_regions(NUTS_to_merge, ['UKC', 'UKD'], 411)
UK_412 = merge_nuts_regions(NUTS_to_merge, ['UKE', 'UKF', 'UKH', 'UKJ'], 412)
UK_413 = merge_nuts_regions(NUTS_to_merge, ['UKG', 'UKK'], 413)

# Greece
EE_450 = merge_nuts_regions(NUTS_to_merge, ['EL51', 'EL52', 'EL53'], 450)
EE_460 = merge_nuts_regions(NUTS_to_merge, ['EL54', 'EL62', 'EL63', 'EL65'], 460)
EE_480 = merge_nuts_regions(NUTS_to_merge, ['EL41', 'EL42', 'EL43', 'EL64'], 480)

# Portugal 
PT_615 = merge_nuts_regions(NUTS_to_merge, ['PT11',	'PT16D', 'PT16E', 'PT16F', 'PT16G', 'PT16H', 'PT16I', 'PT16J'], 615)
PT_630 = merge_nuts_regions(NUTS_to_merge, ['PT17', 'PT16B'], 630)
PT_640 = merge_nuts_regions(NUTS_to_merge, ['PT15', 'PT18'], 640)
PT_650 = merge_nuts_regions(NUTS_to_merge, ['PT20', 'PT30'], 650)

# Finland
FI_670 = merge_nuts_regions(NUTS_to_merge, ['FI1B1', 'FI1C'], 670)
FI_680 = merge_nuts_regions(NUTS_to_merge, ['FI193', 'FI1D1', 'FI1D2', 'FI1D3'], 680)
FI_690 = merge_nuts_regions(NUTS_to_merge, ['FI194', 'FI195', 'FI196', 'FI197', 'FI1D5'], 690)
FI_700 = merge_nuts_regions(NUTS_to_merge, ['FI1D7', 'FI1D8', 'FI1D9'], 700)

# Sweden
SE_710 = merge_nuts_regions(NUTS_to_merge, ['SE1', 'SE2'], 710)
SE_720 = merge_nuts_regions(NUTS_to_merge, ['SE31', 'SE32'], 720)

# Hungary
HU_767 = merge_nuts_regions(NUTS_to_merge, ['HU32', 'HU33'], 767)

# Poland
PL_785 = merge_nuts_regions(NUTS_to_merge, ['PL42', 'PL6'], 785)
PL_790 = merge_nuts_regions(NUTS_to_merge, ['PL22', 'PL41', 'PL51', 'PL52'], 790)
PL_795 = merge_nuts_regions(NUTS_to_merge, ['PL43', 'PL7', 'PL81', 'PL92'], 795)
PL_800 = merge_nuts_regions(NUTS_to_merge, ['PL21', 'PL82', 'PL84'], 800)

# Croatia
HR_862 = merge_nuts_regions(NUTS_to_merge, ['HR02', 'HR05', 'HR06'], 862)

### Concatenate
merge_list = [
    DE_15, UK_411, UK_412, UK_413,         
    EE_450, EE_460, EE_480,                
    PT_615, PT_630, PT_640, PT_650,        
    FI_670, FI_680, FI_690, FI_700,        
    SE_710, SE_720, HU_767, PL_785, 
    PL_790, PL_795, PL_800, HR_862                                 
]

NUTS_merged = gpd.GeoDataFrame(pd.concat(merge_list, ignore_index=True))

In [177]:
##### Clean and combine non-merged geometries

NUTS_no_merge = NUTS_no_merge.merge(FADN_match, on='NUTS_ID', how='left')

NUTS_no_merge = NUTS_no_merge[['FADN_code_2020', 'geometry']]

FADN_geo = gpd.GeoDataFrame(pd.concat([NUTS_no_merge, NUTS_merged], ignore_index=True))

FADN_geo = FADN_geo.merge(FADN_match, on='FADN_code_2020', how='outer')

In [178]:
FADN_geo

Index(['FADN_ID_2020', 'FADN_code_2020', 'FADN_name_2020', 'ISO2', 'NUTS_ID'], dtype='object')

In [179]:
##### Save
FADN_geo.to_file(save_path)

/var/folders/48/ky2jtbmj31bfj15cr5gq480w0000gn/T/ipykernel_74667/3178434704.py:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  FADN_geo.to_file(save_path)
/Users/carinamanitius/Library/Python/3.9/lib/python/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'FADN_code_2020' to 'FADN_code_'
  ogr_write(
/Users/carinamanitius/Library/Python/3.9/lib/python/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'FADN_ID_2020' to 'FADN_ID_20'
  ogr_write(
/Users/carinamanitius/Library/Python/3.9/lib/python/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'FADN_name_2020' to 'FADN_name_'
  ogr_write(
